In [2]:
import os.path
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras import layers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
dataset = r"C:\Users\Sharan Prabhath\Downloads\train"
classes = ["0", "1", "2", "3"]

In [4]:
img_size = 224
training_data = []

In [4]:
def create_training_data():
    for category in classes:
        path = os.path.join(dataset, category)
        class_num=classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass


In [5]:
create_training_data()
print(len(training_data))

12000


In [6]:
random.shuffle(training_data)

In [7]:
X = []  # data/feature
Y = []  # label

for features, label in training_data:
    X.append(features)
    Y.append(label)

In [8]:
x = np.array(X).reshape(-1, img_size, img_size, 3)

In [9]:
x=x/255.0

In [10]:
Y=np.array(Y)

In [13]:
model = tf.keras.applications.MobileNetV2()

In [14]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [15]:
base_input = model.layers[0].input
base_output = model.layers[-2].output
print(base_output)

KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='global_average_pooling2d/Mean:0', description="created by layer 'global_average_pooling2d'")


In [16]:
final_output = layers.Dense(128)(base_output)  ##adding new layer after global_average_pooling
final_output = layers.Activation("relu")(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation("relu")(final_output)
final_output = layers.Dense(4, activation="softmax")(final_output)  ##my classes are 4, classification layer und somit softmax


In [17]:
new_model = keras.Model(inputs=base_input, outputs=final_output)


In [19]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [20]:
new_model.fit(x,Y,epochs=25)

Epoch 1/25
375/375 [==============================] - 1350s 3s/step - loss: 1.0476 - accuracy: 0.5433
Epoch 2/25
375/375 [==============================] - 1240s 3s/step - loss: 0.8924 - accuracy: 0.6280
Epoch 3/25
375/375 [==============================] - 1242s 3s/step - loss: 0.8208 - accuracy: 0.6587
Epoch 4/25
375/375 [==============================] - 1305s 3s/step - loss: 0.7547 - accuracy: 0.6882
Epoch 5/25
375/375 [==============================] - 1413s 4s/step - loss: 0.7000 - accuracy: 0.7106
Epoch 6/25
375/375 [==============================] - 1434s 4s/step - loss: 0.6641 - accuracy: 0.7277
Epoch 7/25
375/375 [==============================] - 1435s 4s/step - loss: 0.6117 - accuracy: 0.7536
Epoch 8/25
375/375 [==============================] - 1304s 3s/step - loss: 0.5714 - accuracy: 0.7739
Epoch 9/25
375/375 [==============================] - 1176s 3s/step - loss: 0.5264 - accuracy: 0.7922
Epoch 10/25
375/375 [==============================] - 1171s 3s/step - loss: 0.466

In [21]:
new_model.save('FER_model.h5')

In [29]:
new_model=tf.keras.models.load_model('FER_model.h5')

In [30]:
data = r"C:\Users\Sharan Prabhath\Downloads\dataset\CK+48"

In [31]:
training_data1= []

In [32]:
def create_training_data1():
    for category in classes:
        path = os.path.join(data, category)
        class_num=classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_data1.append([new_array, class_num])
            except Exception as e:
                pass


In [33]:
create_training_data1()
print(len(training_data1))

501


In [34]:
random.shuffle(training_data1)

In [35]:
X_test = []  # data/feature
Y_test = []  # label

for features, label in training_data1:
    X_test.append(features)
    Y_test.append(label)

In [36]:
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)

In [37]:
X_test=X_test/255.0

In [38]:
Y_test=np.array(Y_test)

In [39]:
print("Accuracy of our model on test data : " , new_model.evaluate(X_test,Y_test)[1]*100 , "%")

16/16 [==============================] - 10s 561ms/step - loss: 0.4442 - accuracy: 0.9202
Accuracy of our model on test data :  92.01596975326538 %


In [40]:
new_model.save('FER_model.h5')